<a href="https://colab.research.google.com/github/newfrogg/data_engineering/blob/review_python/data_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is ELT ?

## Data Engineering for short
It's about the practice of designing and building systems with purposes:
1. collecting,
2. storing,
3. analyzing data at scale.
4. To ensure the highly usable state before being pushed to data scientists, data analysts.

[Coursera-What is Data Engineering](https://www.coursera.org/articles/what-does-a-data-engineer-do-and-how-do-i-become-one)


## What is Data Engineering
"Data engineering is a set of operations aimed at creating ***interfaces and mechanisms for the flow and access of information***. It takes dedicated specialists—data engineers— to maintain data so that it remains available and usable by others. In short, data engineers set up and operate the organization’s data infrastructure, preparing it for further analysis by data analysts and scientists"

-from “Data Engineering and Its Main Concepts” by AlexSoft

**1. Data Engineering Defined**

Data engineering is the development, implementation, and maintenance of systems
and processes that take in raw data and produce high-quality, consistent information that supports downstream use cases, such as analysis and machine learning. Data engineering is the intersection of security, data management, DataOps, data architecture, orchestration, and software engineering. A ***data engineer manages the data engineering lifecycle***, beginning with getting data from source systems and ending with serving data for use cases, such as analysis or machine learning.

**2. The Data Engineering Lifecycle**

- The data engineering lifecycle focus on data itself and shift away the conversation away from technology.
- There are many stages (above and below the iceberge):
    - Generation, Storage, Serving: tasks of data engineering workflow itself.
    - Security, DataOps, Software Engineering: tasks for interact with relevant fields

![Data Engineering Life Cycle](https://github.com/newfrogg/data_engineering/blob/what_is_ELT/images/data_engineering_life_cycle.png?raw=1)

**3. Evolution of the Data Engineer**

*   The roots of data engineering can be traced back to the **data warehousing era (1980s-2000)**, pioneered by figures like Bill Inmon and Ralph Kimball. Early roles like BI engineers and ETL developers focused on building systems for scalable analytics using relational databases and MPP systems-. The rise of the web introduced new data scale challenges that traditional systems struggled with.
*   **Contemporary data engineering emerged in the early 2000s** as companies faced exploding data growth. Innovations from Google (GFS, MapReduce), the open-source Hadoop ecosystem inspired by Google's work, and the advent of public clouds like AWS provided the **foundation for distributed computation and storage** on massive clusters-. This marked the beginning of the "big data" era.
*   The **big data era (2000s-2010s)** saw the rise of the "big data engineer" proficient in software development and low-level infrastructure hacking to manage complex open-source tools like Hadoop and Spark-. While powerful, managing these massive clusters was operationally burdensome and costly, often diverting engineers from delivering business value. The term "big data" has since become a relic as the technology became more accessible.
*   In the **2020s, big data engineers are now simply called data engineers or data lifecycle engineers**, reflecting a shift towards managing the entire data engineering lifecycle rather than low-level infrastructure details-. With greater abstraction and simplification of tools, the focus has moved to higher-value areas like security, data management, DataOps, data architecture, and orchestration. Data engineering has become a discipline of **connecting various modular technologies** to serve business goals.

**4. Data Engineering and Data Science**

- There are many opinions abouth relationship between data engineering and data science. However, for this case, we consider that **data engineering is a separate discipline from data science and analytics**, although they are complementary. Data engineering is described as sitting **upstream** from data science, meaning data engineers are responsible for providing the necessary data inputs for data scientists.

- Considering **Data Science Hierarchy of Needs** published in 2017 by Monica Rogati, which places AI and machine learning at the top, with foundational tasks like data movement, storage, collection, cleansing, and infrastructure at the bottom. The sources state that data scientists often spend a significant majority of their time, estimated at **70% to 80%, on these lower-level tasks** such as gathering, cleaning, and processing data. This occurs because data scientists are typically not trained to engineer production-grade data systems.

![Data Science Hierarchy](https://github.com/newfrogg/data_engineering/blob/what_is_ELT/images/data_science_hierarchy.png?raw=1)

- The core idea is that **data engineers build the solid data foundation** represented by the bottom layers of this hierarchy. By doing so, data engineers enable data scientists to focus their time more effectively on higher-value activities like analysis, experimentation, and machine learning (the top layers of the pyramid). Ultimately, data engineering bridges the gap between acquiring raw data and extracting value from it, playing a vital role in the success of data science in production environments.

[Fundamentals of Data Engineering](https://www.oreilly.com/library/view/fundamentals-of-data/9781098108298/)

## What is ETL ?
ETL is about a *data integration process* including:
1. **Extract** data from legacy system
2. **Transform** and/or clean data to enhance data quality, improve consitency
3. **Load** data into target databases

[IBM_ETL](https://www.ibm.com/think/topics/etl)



### Example: Amazon Web scraping
Working though scraping data about books matching keyword "Data Engineering" from an ecommerce platform - Amazon to practice ETL process with html data.
1. **Extract**: Search for book data from Amazon, staring with searching "Data Engineering" then scraping the data of each webpages like price, ratings, reviews, prices...
2. **Transform**: Pull extracted data to an .csv file for storage and later usage
3. <font color='red'>**Load**</font>: This example doesn't show the target database. So this phase is still left open for being defined later.

Reference: https://github.com/kunal-geeks/amazon-web-scraping/tree/main

In [1]:
# Import Libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import json

In [2]:
# Support Function
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("div",attrs={'class':'a-section aok-hidden twister-plus-buying-options-price-data'}).string.strip()
        price = json.loads(price)
        price = price["desktop_buybox_group_1"][0]["displayPrice"]
        # price = soup.find("span", attrs={'class':'a-size-base a-color-price a-color-price'}).string.strip()
    except:
        price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

# Function to extract pages_count (On going fixing - the content is not stable => can't catch correctly each time)
def get_pages(soup):
    try:
        # method 1
        pages = soup.find("ul", attrs={'class':'a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list'})
        if pages:
            pages = pages.find_all("li")[4]

        if pages:
            pages = pages.find("span")

        if pages:
            pages = pages.find_all("span")[1].string.strip()
        pages = soup.find("a",attrs={'aria-label':re.compile('Print length.*')})
        # method 2
        # pages = soup.find_all("a",attrs={'aria-description':'Popover with more information about the attribute'})
        # if pages:
        #     pages = pages[0]
        # print(pages)
        # pages = pages.get('aria-label').string.strip()

    except AttributeError:
        pages = "Not Available"
    return pages

# Function to extract main author
def get_author(soup):
    try:
        author = soup.find("div", attrs={'id':'bylineInfo', 'class':'a-section a-spacing-micro bylineHidden feature'})
        author = author.find("span")
        author = author.find("a").string.strip()
    except AttributeError:
        author = "Not Available"
    return author


In [3]:
if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=data+engineering&i=stripbooks-intl-ship"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[], "authors":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
        d['authors'].append(get_author(new_soup))
        # break


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df.replace({'title':''},np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

/tmp/ipython-input-3-2588403658.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [4]:
amazon_df

,title,price,rating,reviews,availability,authors
0,Fundamentals of Data Engineering: Plan and Bui...,,4.7 out of 5 stars,703 ratings,Not Available,Joe Reis
1,Data Engineering Design Patterns: Recipes for ...,$63.50,4.4 out of 5 stars,2 ratings,In Stock,Bartosz Konieczny
2,AI Engineering: Building Applications with Fou...,$57.74,4.4 out of 5 stars,217 ratings,In Stock,Chip Huyen
3,Data Pipelines Pocket Reference: Moving and Pr...,$16.93,4.4 out of 5 stars,400 ratings,In Stock,James Densmore
4,Designing Data-Intensive Applications: The Big...,$43.23,4.4 out of 5 stars,"5,200 ratings",In Stock,Martin Kleppmann
5,Financial Data Engineering: Design and Build D...,$49.20,4.4 out of 5 stars,8 ratings,In Stock,Tamer Khraisha
6,Ace the Data Engineering Interview: Questions ...,$29.99,5.0 out of 5 stars,3 ratings,In Stock,Sean Coyne
7,Data Engineering Best Practices: Architect rob...,$35.99,5.0 out of 5 stars,4 ratings,In Stock,Richard J. Schiller
8,Databricks Certified Data Engineer Associate S...,$64.59,4.4 out of 5 stars,11 ratings,In Stock,Derar Alhussein
9,Data Engineering with Python: Work with massiv...,$41.99,4.1 out of 5 stars,146 ratings,In Stock,Paul Crickard


# Review Python
Below is a 1-2 week project-based learning plan to review Python basics up to lists and tuples, tailored for data engineering and data analysis. The project involves building **a simple data pipeline** to process and analyze a dataset of customer transactions, reinforcing Python fundamentals while introducing data-related concepts.

In [5]:
def create_sample_csv(file_path):
    data = [
        "transaction_id,customer_name,amount,date,category",
        "1,John Doe,50.25,2025-06-01,Food",
        "2,Jane Smith,120.50,2025-06-02,Electronics",
        "3,Alice Johnson,30.00,2025-06-03,Clothing",
        "4,John Doe,75.80,2025-06-03,Food",
        "5,Bob Wilson,200.00,2025-06-04,Electronics",
        "6,Jane Smith,15.99,2025-06-05,Books",
        "7,Alice Johnson,45.60,2025-06-06,Clothing",
        "8,John Doe,-10.00,2025-06-07,Food",
        "9,Emma Davis,80.00,2025-06-08,Electronics",
        "10,Bob Wilson,25.50,2025-06-09,Books",
        "11,Jane Smith,60.75,2025-06-10,Food",
        "12,Invalid User,abc,2025-06-11,Clothing",
        "13,Emma Davis,90.20,2025-06-12,Electronics",
        "14,John Doe,35.40,2025-06-13,Books",
        "15,Alice Johnson,70.00,2025-06-14,Clothing"
    ]
    with open(file_path, 'w') as file:
        for line in data:
            file.write(line + '\n')
    print(f"Sample CSV created at {file_path} as below")

create_sample_csv('transactions.csv')
!cat transactions.csv | column -t -s ","

Sample CSV created at transactions.csv as below
transaction_id  customer_name  amount  date        category
1               John Doe       50.25   2025-06-01  Food
2               Jane Smith     120.50  2025-06-02  Electronics
3               Alice Johnson  30.00   2025-06-03  Clothing
4               John Doe       75.80   2025-06-03  Food
5               Bob Wilson     200.00  2025-06-04  Electronics
6               Jane Smith     15.99   2025-06-05  Books
7               Alice Johnson  45.60   2025-06-06  Clothing
8               John Doe       -10.00  2025-06-07  Food
9               Emma Davis     80.00   2025-06-08  Electronics
10              Bob Wilson     25.50   2025-06-09  Books
11              Jane Smith     60.75   2025-06-10  Food
12              Invalid User   abc     2025-06-11  Clothing
13              Emma Davis     90.20   2025-06-12  Electronics
14              John Doe       35.40   2025-06-13  Books
15              Alice Johnson  70.00   2025-06-14  Clothing


In [6]:
!rm -rf transactions.*
!echo "transaction_id,customer_name,amount,date,category" >> transactions.csv
!echo "1,John Doe,50.25,2025-06-01,Food" >> transactions.csv
!echo "2,Jane Smith,120.50,2025-06-02,Electronics" >> transactions.csv
!echo "3,Alice Johnson,30.00,2025-06-03,Clothing" >> transactions.csv
!echo "4,John Doe,75.80,2025-06-03,Food" >> transactions.csv
!echo "5,Bob Wilson,200.00,2025-06-04,Electronics" >> transactions.csv
!echo "6,Jane Smith,15.99,2025-06-05,Books" >> transactions.csv
!echo "7,Alice Johnson,45.60,2025-06-06,Clothing" >> transactions.csv
!echo "8,John Doe,-10.00,2025-06-07,Food" >> transactions.csv
!echo "9,Emma Davis,80.00,2025-06-08,Electronics" >> transactions.csv
!echo "10,Bob Wilson,25.50,2025-06-09,Books" >> transactions.csv
!echo "11,Jane Smith,60.75,2025-06-10,Food" >> transactions.csv
!echo "12,Invalid User,abc,2025-06-11,Clothing" >> transactions.csv
!echo "13,Emma Davis,90.20,2025-06-12,Electronics" >> transactions.csv
!echo "14,John Doe,35.40,2025-06-13,Books" >> transactions.csv
!echo "15,Alice Johnson,70.00,2025-06-14,Clothing" >> transactions.csv
!echo "===================================================================="
!echo "======================== Transaction Table ========================="
!echo "===================================================================="
!cat transactions.csv | column -t -s "," >> transactions.table
!cat transactions.table

======================== Transaction Table =========================
transaction_id  customer_name  amount  date        category
1               John Doe       50.25   2025-06-01  Food
2               Jane Smith     120.50  2025-06-02  Electronics
3               Alice Johnson  30.00   2025-06-03  Clothing
4               John Doe       75.80   2025-06-03  Food
5               Bob Wilson     200.00  2025-06-04  Electronics
6               Jane Smith     15.99   2025-06-05  Books
7               Alice Johnson  45.60   2025-06-06  Clothing
8               John Doe       -10.00  2025-06-07  Food
9               Emma Davis     80.00   2025-06-08  Electronics
10              Bob Wilson     25.50   2025-06-09  Books
11              Jane Smith     60.75   2025-06-10  Food
12              Invalid User   abc     2025-06-11  Clothing
13              Emma Davis     90.20   2025-06-12  Electronics
14              John Doe       35.40   2025-06-13  Books
15              Alice Johnson  70.00   2025-

## Project Overview
- **Objective**: Create a Python script that reads a CSV file of customer transactions, cleans the data, and generates basic analytical insights (e.g., total sales, top customers).
- **Dataset**: Use a sample CSV file with columns: `transaction_id`, `customer_name`, `amount`, `date`, `category`.
- **Tools**: Python, `csv` module (standard library).
- **Duration**: 1-2 weeks, ~2-3 hours/day.

## Week 1: Python Basics and Data Ingestion



### Day 1: Setup and Python Basics
- **Topics**: Variables, data types (int, float, str, bool), basic operations, input/output.
- **Task**: Set up Python environment, create a sample CSV file, and write a script to read it.
- **Activity**:
  - Create a CSV file `transactions.csv` with 10-15 rows (e.g., `1,John Doe,50.25,2025-06-01,Food`).
  - Write a script to open and print the file content.
- **Code**:
  ```python
  # Read and print CSV file
  with open('transactions.csv', 'r') as file:
      for line in file:
          print(line.strip())
  ```

**Close Files**

It is a good practice to always close the file when you are done with it.

If you are not using the with statement, you must write a close statement in order to close the file:
  ```python
  f = open("demofile.txt")
  print(f.readline())
  f.close()
  ```

**Strip function**

The strip() method removes any leading, and trailing whitespaces.

Leading means at the beginning of the string, trailing means at the end.

You can specify which character(s) to remove, if not, any whitespaces will be removed.

**Syntax**: *string.strip(characters)*

**Example**:
```python
txt = ",,,,,rrttgg.....banana....rrr"
x = txt.strip(",.grt")
print(x)
```


In [7]:
with open('transactions.csv', 'r') as file:
    print("case 1: w/o strip function")
    for line in file:
        print(line)

case 1: w/o strip function
transaction_id,customer_name,amount,date,category

1,John Doe,50.25,2025-06-01,Food

2,Jane Smith,120.50,2025-06-02,Electronics

3,Alice Johnson,30.00,2025-06-03,Clothing

4,John Doe,75.80,2025-06-03,Food

5,Bob Wilson,200.00,2025-06-04,Electronics

6,Jane Smith,15.99,2025-06-05,Books

7,Alice Johnson,45.60,2025-06-06,Clothing

8,John Doe,-10.00,2025-06-07,Food

9,Emma Davis,80.00,2025-06-08,Electronics

10,Bob Wilson,25.50,2025-06-09,Books

11,Jane Smith,60.75,2025-06-10,Food

12,Invalid User,abc,2025-06-11,Clothing

13,Emma Davis,90.20,2025-06-12,Electronics

14,John Doe,35.40,2025-06-13,Books

15,Alice Johnson,70.00,2025-06-14,Clothing



In [8]:
with open('transactions.csv', 'r') as file:
    print("case 2: with strip function")
    for line in file:
        print(line.strip())


case 2: with strip function
transaction_id,customer_name,amount,date,category
1,John Doe,50.25,2025-06-01,Food
2,Jane Smith,120.50,2025-06-02,Electronics
3,Alice Johnson,30.00,2025-06-03,Clothing
4,John Doe,75.80,2025-06-03,Food
5,Bob Wilson,200.00,2025-06-04,Electronics
6,Jane Smith,15.99,2025-06-05,Books
7,Alice Johnson,45.60,2025-06-06,Clothing
8,John Doe,-10.00,2025-06-07,Food
9,Emma Davis,80.00,2025-06-08,Electronics
10,Bob Wilson,25.50,2025-06-09,Books
11,Jane Smith,60.75,2025-06-10,Food
12,Invalid User,abc,2025-06-11,Clothing
13,Emma Davis,90.20,2025-06-12,Electronics
14,John Doe,35.40,2025-06-13,Books
15,Alice Johnson,70.00,2025-06-14,Clothing


### Day 2: Lists and Basic Data Structures
- **Topics**: Lists, indexing, slicing, list methods (append, remove, sort).
- **Task**: Parse CSV rows into a list of lists, extract specific columns.
- **Activity**:
  - Modify the script to store each row as a list in a master list.
  - Print all transaction amounts (column 2).
- **Code**:
  ```python
  transactions = []
  with open('transactions.csv', 'r') as file:
      for line in file:
          row = line.strip().split(',')
          transactions.append(row)
  amounts = [float(row[2]) for row in transactions[1:]]  # Skip header
  print(amounts)
  ```

In [9]:
transactions = []
with open('transactions.csv', 'r') as file:
    for line in file:
        row = line.strip().split(',')
        transactions.append(row)
amounts = [float(row[2]) for row in transactions[1:]]  # Skip header
print(amounts)

ValueError: could not convert string to float: 'abc'

**This above errors caused by invalid value in amount column**.

We try to apply try and catch to cover this unexpected behavior as below code.

![Invalid amount data](https://github.com/newfrogg/data_engineering/blob/review_python/images/invalid_amount.png?raw=1)

In [10]:
transactions = []
with open('transactions.csv', 'r') as file:
    for line in file:
        row = line.strip().split(',')
        transactions.append(row)

amount_original = []
amount_modified = []
## Try retrieve the amount value and convert to true format number.
for index, row in enumerate(transactions[1:]):
    amount_original.append(row[2])
    try:
        print(f"Infomation: Convert amount {row[2]} from String to Float successfully")
        amount_modified.append(float(row[2]))
    except ValueError:
        print(f"Warning: Invalid amount: {row[2]} => Change to default value (0)")
        amount_modified.append(float(0.0))

!echo "<<< Original column 'amount'"
print(amount_original)
!echo ">>> Modified column 'amount'"
print(amount_modified)

Infomation: Convert amount 50.25 from String to Float successfully
Infomation: Convert amount 120.50 from String to Float successfully
Infomation: Convert amount 30.00 from String to Float successfully
Infomation: Convert amount 75.80 from String to Float successfully
Infomation: Convert amount 200.00 from String to Float successfully
Infomation: Convert amount 15.99 from String to Float successfully
Infomation: Convert amount 45.60 from String to Float successfully
Infomation: Convert amount -10.00 from String to Float successfully
Infomation: Convert amount 80.00 from String to Float successfully
Infomation: Convert amount 25.50 from String to Float successfully
Infomation: Convert amount 60.75 from String to Float successfully
Infomation: Convert amount abc from String to Float successfully
Infomation: Convert amount 90.20 from String to Float successfully
Infomation: Convert amount 35.40 from String to Float successfully
Infomation: Convert amount 70.00 from String to Float success

### Day 3: Conditionals and Data Cleaning
- **Topics**: If-else statements, comparison operators, handling missing data.
- **Task**: Identify and handle invalid entries (e.g., negative amounts, empty fields).
- **Activity**:
  - Add checks to skip rows with invalid amounts (e.g., negative or non-numeric).
  - Log invalid rows to a list.
  
We try to separate transaction have wrong value. By filtering data from ***amount column***, we eliminate 2 invalid out of 15 column at all.

In [11]:
invalid_rows = []
valid_transactions = []
with open('transactions.csv', 'r') as file:
    next(file)  # Skip header
    for line in file:
        row = line.strip().split(',')
        try:
            amount = float(row[2])
            if amount < 0:
                invalid_rows.append(row)
            else:
                valid_transactions.append(row)
        except ValueError:
            invalid_rows.append(row)
print(f"Valid transactions: {len(valid_transactions)} of {len(valid_transactions) + len(invalid_rows)}")
print(f"Invalid transactions: {len(invalid_rows)}")

print(invalid_rows)


Valid transactions: 13 of 15
Invalid transactions: 2
[['8', 'John Doe', '-10.00', '2025-06-07', 'Food'], ['12', 'Invalid User', 'abc', '2025-06-11', 'Clothing']]


In [12]:
print(invalid_rows)

[['8', 'John Doe', '-10.00', '2025-06-07', 'Food'], ['12', 'Invalid User', 'abc', '2025-06-11', 'Clothing']]


In [13]:
[print(row) for row in invalid_rows]

['8', 'John Doe', '-10.00', '2025-06-07', 'Food']
['12', 'Invalid User', 'abc', '2025-06-11', 'Clothing']


[None, None]

The **issue** you're describing arises because the list **comprehension print(row) for row in invalid_rows** prints each row but also returns a list of the results of the print() function, which is None for each call. **This is why you see [None, None] in the output after the rows are printed.**

#### **Why does it print None, None?**
The print(row) function outputs the row to the console and returns None.
The list comprehension creates a list of these return values (None for each row).
```python
[print(row) for row in invalid_rows]
```
For your invalid_rows with two rows, the comprehension results in [None, None], which is then displayed as part of the output.

Solution:
1. Use an explicit for loop
```python
for row in invalid_rows:
    print(row)
```
2. Use a side-effect-free comprehension
```python
_ = [print(row) for row in invalid_rows]
```

In [14]:
_= [print(row) for row in invalid_rows]

['8', 'John Doe', '-10.00', '2025-06-07', 'Food']
['12', 'Invalid User', 'abc', '2025-06-11', 'Clothing']


A side-effect-free comprehension focuses solely on transforming or filtering data to create a new collection (list, set, or dictionary) **without performing external operations like printing**. It adheres to the principle of functional programming, where operations are predictable and don’t affect external state.

For example, a side-effect-free version of your comprehension would avoid print() and simply collect or transform the rows:

#### **What is a Side Effect?**
A side effect occurs when a function or operation does something beyond returning a value. In contrast, a pure function or operation has no side effects—it only computes and returns a value based on its inputs.



### Day 4: Loops and Data Aggregation
- **Topics**: For loops, while loops, iterating over lists.
- **Task**: Calculate total sales and count transactions per category.
- **Activity**:
  - Use a loop to sum transaction amounts.
  - Create a list to store category counts.

Using with allowing automatically close file after completing execute code inside this scope.
```python
with open('$file_name','mode) as file:
    next(file) # Skip header
```


In [15]:
total_sales = 0
category_counts = []
categories = []
with open('transactions.csv', 'r') as file:
    next(file)  # Skip header
    for line in file:
        row = line.strip().split(',')
        try:
            amount = float(row[2])
            if amount >= 0:
                total_sales += amount
                category = row[4]
                if category not in categories:
                    categories.append(category)
                    category_counts.append(1)
                else:
                    index = categories.index(category)
                    category_counts[index] += 1
        except ValueError:
            continue
print(f"Total sales: ${total_sales:.2f}")
print("Category counts:", list(zip(categories, category_counts)))

Total sales: $899.99
Category counts: [('Food', 3), ('Electronics', 4), ('Clothing', 3), ('Books', 3)]


- **zip function help zip the lists** to each other that catch by minlenght of list. Then set list to make it an object readable. Example:
```python
a = [1,2,3]
b = [2,45]
c = [312, 121, 123,1 ,12]
list(zip(a,b,c))
# result>> [(1, 2, 312), (2, 45, 121)]
```
- A new approach that help clean and minimize that simple code. Obviously, when we control the Value Error of data
```python
total_sales = 0
total_sales = sum(float(row[2]) for row in valid_transactions)
print(f"Total sales: ${total_sales:.2f}")
# result>> Total sales: $899.99
```

In [16]:
a = [1,2,3]
b = [2,45]
c = [312, 121, 123,1 ,12]
list(zip(a,b,c))

[(1, 2, 312), (2, 45, 121)]

##### **An alternative using pandas library**
For this example, we can see the powerful of library, that cover corner cases, catch exceptions.

In [17]:
import pandas as pd

# Read CSV into a DataFrame
df = pd.read_csv('transactions.csv')

# Convert 'Amount' to numeric, coercing errors to NaN
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

# Filter valid rows (non-negative amounts, non-NaN)
valid_df = df[df['amount'] >= 0]

# Calculate total sales
total_sales = valid_df['amount'].sum()

# Get category frequencies
category_freq = valid_df['category'].value_counts().to_dict()

print(f"Total sales: ${total_sales:.2f}")
print("Category counts:", list(category_freq.items()))

Total sales: $899.99
Category counts: [('Electronics', 4), ('Food', 3), ('Clothing', 3), ('Books', 3)]


### Day 5: Functions and Modularity
- **Topics**: Defining functions, parameters, return values.
- **Task**: Refactor code into functions for reusability.
- **Activity**:
  - Write functions for reading CSV, cleaning data, and calculating totals.
  - Call functions in a main script.


In [18]:
def read_csv(file_path):
      transactions = []
      with open(file_path, 'r') as file:
          next(file)  # Skip header
          for line in file:
              transactions.append(line.strip().split(','))
      return transactions

def clean_data(transactions):
    valid = []
    invalid = []
    for row in transactions:
        try:
            amount = float(row[2])
            if amount >= 0:
                valid.append(row)
            else:
                invalid.append(row)
        except ValueError:
            invalid.append(row)
    return valid, invalid

def calculate_totals(valid_transactions):
    total_sales = sum(float(row[2]) for row in valid_transactions)
    return total_sales

# Main script
## Reading data from CSV file into 2-dimension list
transactions = read_csv('transactions.csv')
## Cleanning invalid rows (contain string in numeric cells)
valid, invalid = clean_data(transactions)
## Sum of total_sales (valid)
total_sales = calculate_totals(valid)
print(f"Total sales: ${total_sales:.2f}")
print(f"Invalid rows: {len(invalid)}")

Total sales: $899.99
Invalid rows: 2


## Week 2: Tuples, Analysis, and Pipeline Completion

### Day 6: Tuples and Immutable Data
- **Topics**: Tuples, tuple operations, use cases.
- **Task**: Store cleaned transactions as tuples for immutability.
- **Activity**:
  - Convert valid transactions to a list of tuples.
  - Verify tuple immutability by attempting to modify one.

In Python, a tuple is an immutable, ordered collection of items that can store elements of any data type (e.g., numbers, strings, lists, or even other tuples). **Tuples are similar to lists but cannot be modified after creation**, making them useful for representing fixed collections of data.




In [19]:
def clean_data_to_tuples(transactions):
    valid = []
    invalid = []
    for row in transactions:
        try:
            amount = float(row[2])
            if amount >= 0:
                valid.append((row[0], row[1], amount, row[3], row[4]))
            else:
                invalid.append(row)
        except ValueError:
            invalid.append(row)
    return valid, invalid

transactions = read_csv('transactions.csv')
valid_tuples, invalid = clean_data_to_tuples(transactions)
print("First transaction:", valid_tuples[2])
print("Valid transactions under tuple form")
_ = [print(valid) for valid in valid_tuples]


First transaction: ('3', 'Alice Johnson', 30.0, '2025-06-03', 'Clothing')
Valid transactions under tuple form
('1', 'John Doe', 50.25, '2025-06-01', 'Food')
('2', 'Jane Smith', 120.5, '2025-06-02', 'Electronics')
('3', 'Alice Johnson', 30.0, '2025-06-03', 'Clothing')
('4', 'John Doe', 75.8, '2025-06-03', 'Food')
('5', 'Bob Wilson', 200.0, '2025-06-04', 'Electronics')
('6', 'Jane Smith', 15.99, '2025-06-05', 'Books')
('7', 'Alice Johnson', 45.6, '2025-06-06', 'Clothing')
('9', 'Emma Davis', 80.0, '2025-06-08', 'Electronics')
('10', 'Bob Wilson', 25.5, '2025-06-09', 'Books')
('11', 'Jane Smith', 60.75, '2025-06-10', 'Food')
('13', 'Emma Davis', 90.2, '2025-06-12', 'Electronics')
('14', 'John Doe', 35.4, '2025-06-13', 'Books')
('15', 'Alice Johnson', 70.0, '2025-06-14', 'Clothing')


In [20]:
# Try modifying tuple (will raise error)
try:
    valid_tuples[0][2] = 100
except TypeError as e:
    print("Error:", e)

Error: 'tuple' object does not support item assignment



### Day 7: Data Analysis with Lists and Tuples
- **Topics**: Sorting lists, filtering, list comprehensions.
- **Task**: Generate insights (e.g., top 3 customers by total spend).
- **Activity**:
  - Create a list of (customer_name, total_amount) tuples.
  - Sort and extract top 3 customers.


In [21]:
transactions = read_csv('transactions.csv')
valid_tuples, _ = clean_data_to_tuples(transactions)
_ = [print(valid) for valid in valid_tuples]

('1', 'John Doe', 50.25, '2025-06-01', 'Food')
('2', 'Jane Smith', 120.5, '2025-06-02', 'Electronics')
('3', 'Alice Johnson', 30.0, '2025-06-03', 'Clothing')
('4', 'John Doe', 75.8, '2025-06-03', 'Food')
('5', 'Bob Wilson', 200.0, '2025-06-04', 'Electronics')
('6', 'Jane Smith', 15.99, '2025-06-05', 'Books')
('7', 'Alice Johnson', 45.6, '2025-06-06', 'Clothing')
('9', 'Emma Davis', 80.0, '2025-06-08', 'Electronics')
('10', 'Bob Wilson', 25.5, '2025-06-09', 'Books')
('11', 'Jane Smith', 60.75, '2025-06-10', 'Food')
('13', 'Emma Davis', 90.2, '2025-06-12', 'Electronics')
('14', 'John Doe', 35.4, '2025-06-13', 'Books')
('15', 'Alice Johnson', 70.0, '2025-06-14', 'Clothing')


In [22]:
def get_top_customers(valid_tuples, n=3, ascending=True):
    customer_totals = []
    customers = []
    for transaction in valid_tuples:
        customer, amount = transaction[1], transaction[2]
        if customer not in customers:
            customers.append(customer)
            customer_totals.append(amount)
        else:
            index = customers.index(customer)
            customer_totals[index] += amount
    customer_summary = list(zip(customers, customer_totals))
    get_first_element_of_tuple = lambda x: x[1]
    topN_customer = sorted(customer_summary, key=get_first_element_of_tuple, reverse=ascending)[:n]
    return topN_customer[:n]
### ascending: tăng dần
top_customers = get_top_customers(valid_tuples, n=3, ascending=False)

print("Top 3 customers by spend:")
for customer, total in top_customers:
    print(f"{customer}: ${total:.2f}")

Top 3 customers by spend:
Alice Johnson: $145.60
John Doe: $161.45
Emma Davis: $170.20


**A lambda function** in Python is a small, anonymous (unnamed) function defined using the lambda keyword. It’s used for short, one-off operations, especially in places like key parameters for sorting or filtering.

Syntax
```python
lambda arguments: expression
```
- Arguments: Inputs to the function (e.g., x).
- Expression: A single expression evaluated and returned (e.g., x[1]).
- Example: ```lambda x: x[1]``` takes a tuple x and returns its second element.
##### Difference Between lambda and def Keyword
lambda is concise but less powerful than def when handling complex logic. Let's take a look at short comparison between the two:

| Feature           | `lambda` Function                    | Regular Function (`def`)               |
| ----------------- | ------------------------------------ | -------------------------------------- |
| **Definition**    | Single expression with `lambda`.     | Multiple lines of code.                |
| **Name**          | Anonymous (or named if assigned).    | Must have a name.                      |
| **Statements**    | Single expression only.              | Can include multiple statements.       |
| **Documentation** | Cannot have a docstring.             | Can include docstrings.                |
| **Reusability**   | Best for short, temporary functions. | Better for reusable and complex logic. |

In [27]:
print("====> Example 1: Sign of x")
sign = lambda x: "Positive" if x > 0 else "Negative" if x < 0 else "Zero"
print(f"Sign of 10 is {sign(10)}")
print(f"Sign of -10 is {sign(-10)}")
print(f"Sign of 0 is {sign(0)}")
print("====> Example 2: Uppercase")
upperString = lambda sttr: sttr.upper()
print(f"Uppercase of 'hello' is {upperString('hello')}")
print("====> Example 3: Relu Function")
Relu = lambda x: x if x > 0 else 0
print(f"Relu of 10 is {Relu(10)}")
print(f"Relu of -10 is {Relu(-10)}")
print("====> Example 4: Multi statements")
calculate = lambda x, y, op: x + y if op == "add" else x - y if op == "sub" else "Invalid operation"
print(f"2 added 3 equal to {calculate(2,3,op='add')}")
print(f"2 subtracted 3 equal to {calculate(2,3,op='sub')}")


====> Example 1: Sign of x
Sign of 10 is Positive
Sign of -10 is Negative
Sign of 0 is Zero
====> Example 2: Uppercase
Uppercase of 'hello' is HELLO
====> Example 3: Relu Function
Relu of 10 is 10
Relu of -10 is 0
====> Example 4: Multi statements
2 added 3 equal to 5
2 subtracted 3 equal to -1
2 subtracted 3 equal to Invalid operation


###### Usage with filter/map function()
[GeeksforGeeks - Python Lambda Functions](https://www.geeksforgeeks.org/python/python-lambda-anonymous-functions-filter-map-reduce/)

In [24]:
n = [1, 2, 3, 4, 5, 6]
isEven = lambda x: x % 2 == 0
even = filter(isEven, n)
print(f"1. Filtering even number only from {n}:\n>>> {list(even)}")

a = [1, 2, 3, 4]
double = lambda x: x * 2
b = map(double, a)
print(f"2. Map doubling each elements from {a}:\n>>> {list(b)}")

1. Filtering even number only from [1, 2, 3, 4, 5, 6]:
>>> [2, 4, 6]
2. Map doubling each elements from [1, 2, 3, 4]:
>>> [2, 4, 6, 8]


###### Others: usage with reduce function()

In [25]:
from functools import reduce

# Example: Find the product of all numbers in a list
a = [1, 2, 3, 4]
multi = lambda x, y: x * y
b = reduce(multi, a)
print(b)

24


### Day 8: Error Handling and Robustness
- **Topics**: Try-except, custom error messages.
- **Task**: Add error handling for file operations and data parsing.
- **Activity**:
  - Handle missing file errors and malformed CSV rows.
  - Test with a broken CSV file.
  

In [26]:
def clean_data_to_tuples(transactions):
    valid = []
    invalid = []
    for row in transactions:
        try:
            amount = float(row[2])
            if amount >= 0:
                valid.append((row[0], row[1], amount, row[3], row[4]))
            else:
                invalid.append(row)
        ## Handle get value not as expected (i.e: String instead of numeric)
        except ValueError:
            invalid.append(row)
    return valid, invalid

def read_csv_safe(file_path):
    try:
        with open(file_path, 'r') as file:
            transactions = [line.strip().split(',') for line in file]
            ### Handle case of empty file
            if not transactions:
                raise ValueError("Empty CSV file")
            return transactions
    ## Handle missing file in case cannot open file with file_path as filled
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found")
        return []
    ## Handle any errors while open for reading file
    except Exception as e:
        print(f"Error reading file: {e}")
        return []
transactions = read_csv_safe('transactions.csv')
if transactions:
    valid_tuples, invalid = clean_data_to_tuples(transactions[1:])  # Skip header
    print(f"Processed {len(valid_tuples)} valid transactions")


Processed 13 valid transactions


### Day 9: Final Pipeline Integration
- **Topics**: Combining functions, script structure.
- **Task**: Build a complete pipeline script.
- **Activity**:
  - Integrate all functions into a single script.
  - Output results to console and a summary file.

- This diagrama visualize the workflow of this project in form of a data pipeline.
<center>
 <img src='https://github.com/newfrogg/data_engineering/blob/review_python/images/data_pipeline.png?raw=1' width='250%' />
</center>



In [29]:
def read_csv_safe(file_path):
    try:
        with open(file_path, 'r') as file:
            transactions = [line.strip().split(',') for line in file]
            if not transactions:
                raise ValueError("Empty CSV file")
            return transactions
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found")
        return []
    except Exception as e:
        print(f"Error reading file: {e}")
        return []
def clean_data_to_tuples(transactions):
    valid = []
    invalid = []
    for row in transactions:
        try:
            amount = float(row[2])
            if amount >= 0:
                valid.append((row[0], row[1], amount, row[3], row[4]))
            else:
                invalid.append(row)
        except (ValueError, IndexError):
            invalid.append(row)
    return valid, invalid
def calculate_totals(valid_tuples):
    total_sales = sum(t[2] for t in valid_tuples)
    return total_sales
def get_top_customers(valid_tuples, n=3):
    customer_totals = []
    customers = []
    for t in valid_tuples:
        customer = t[1]
        amount = t[2]
        if customer not in customers:
            customers.append(customer)
            customer_totals.append(amount)
        else:
            index = customers.index(customer)
            customer_totals[index] += amount
    customer_summary = [(c, t) for c, t in zip(customers, customer_totals)]
    customer_summary.sort(key=lambda x: x[1], reverse=True)
    return customer_summary[:n]
def write_summary(file_path, total_sales, top_customers, invalid_count):
    with open(file_path, 'w') as file:
        file.write(f"Transaction Summary\n")
        file.write(f"Total Sales: ${total_sales:.2f}\n")
        file.write(f"Invalid Transactions: {invalid_count}\n")
        file.write(f"Top {len(top_customers)} Customers:\n")
        for customer, total in top_customers:
            file.write(f"{customer}: ${total:.2f}\n")
# Main pipeline
def run_pipeline(input_file, output_file):
    transactions = read_csv_safe(input_file)
    if not transactions:
        return
    valid_tuples, invalid = clean_data_to_tuples(transactions[1:])  # Skip header
    total_sales = calculate_totals(valid_tuples)
    top_customers = get_top_customers(valid_tuples)
    write_summary(output_file, total_sales, top_customers, len(invalid))
    print(f"Pipeline complete. Summary written to {output_file}")
# Run
run_pipeline('transactions.csv', 'summary.txt')
!cat summary.txt

Pipeline complete. Summary written to summary.txt
Transaction Summary
Total Sales: $899.99
Invalid Transactions: 2
Top 3 Customers:
Bob Wilson: $225.50
Jane Smith: $197.24
Emma Davis: $170.20




### Day 10: Testing and Extensions
- **Topics**: Testing code, debugging.
- **Task**: Test the pipeline and add a new feature.
- **Activity**:
  - Test with different CSV files (e.g., missing columns, large data).
  - Add a function to filter transactions by date range.
- **Extension Code**:
  ```python
  def filter_by_date(valid_tuples, start_date, end_date):
      from datetime import datetime
      filtered = []
      for t in valid_tuples:
          trans_date = datetime.strptime(t[3], '%Y-%m-%d')
          if start_date <= trans_date <= end_date:
              filtered.append(t)
      return filtered

  # Example usage in pipeline
  from datetime import datetime
  start = datetime(2025, 6, 1)
  end = datetime(2025, 6, 15)
  filtered_tuples = filter_by_date(valid_tuples, start, end)
  print(f"Transactions from {start.date()} to {end.date()}: {len(filtered_tuples)}")
  ```

---

## Deliverables
- A complete Python script (`pipeline.py`) that runs the data pipeline.
- A sample `transactions.csv` file.
- A `summary.txt` file with analysis results.
- (Optional) A short report (in comments or separate file) explaining the pipeline.

## Learning Outcomes
- **Python Basics**: Master variables, data types, conditionals, loops, functions.
- **Lists and Tuples**: Understand mutable vs. immutable data structures.
- **Data Engineering**: Learn data ingestion, cleaning, and pipeline design.
- **Data Analysis**: Perform basic aggregations and generate insights.

## Next Steps
- Explore Python dictionaries for more complex aggregations.
- Learn `pandas` for advanced data manipulation.
- Add database integration (e.g., SQLite) for data engineering.

